In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
import math, copy, time
from collections import Counter, OrderedDict
import re
from copy import deepcopy
import pandas as pd
import numpy as np
from torch.nn.utils.rnn import PackedSequence,pack_padded_sequence
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

MIN_LENGTH = 3
MAX_LENGTH = 40

In [3]:
USE_CUDA = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor
USE_CUDA

False

In [4]:
torch.manual_seed(1111)

In [0]:
# 安装 PyDrive 操作库，该操作每个 notebook 只需要执行一次
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 授权登录，仅第一次的时候会鉴权
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
# 列出根目录的所有文件
# "q" 查询条件教程详见：https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))

title: data, id: 16zfnt8Cnuznzwj2RHGrxvyfyA6iQPRXx, mimeType: application/vnd.google-apps.folder
title: Colab Notebooks, id: 1DDn16_H4dL75WfkYDpWNxRzqJkH6uvsk, mimeType: application/vnd.google-apps.folder
title: 02 - email-templates.png, id: 0BzICsNHMK-GFdnZhNFBoVXdobkhBa3h2eUtwbXY1ek15Sk53, mimeType: image/png
title: ATEC-Alipay, id: 1UyfXNCoPHh73E4jO-pgombXQShfYkzAVgGfSUnU7OAE, mimeType: application/vnd.google-apps.document
title: The Annotated _Attention is All You Need_.ipynb, id: 1zIkaTyyRtMY0qRTuUxe9J10xR3S9J5Xc, mimeType: application/vnd.google.colab
title: Copy of mixture_of_softmaxes.ipynb, id: 1HiJvBimCdWvFP7SoLs9bWNR-JbCQSRsG, mimeType: application/vnd.google.colab
title: Copy of Hello, Colaboratory, id: 0BzICsNHMK-GFcFBrQkV2UXpMVGM, mimeType: application/vnd.google.colab
title: 2017 Calendar, id: 1jw6VChc9SGayYuK5Zmy2PjaiwWM0tzGFrPG9eatsSNw, mimeType: application/vnd.google-apps.spreadsheet
title: Getting started, id: 0BzICsNHMK-GFc3RhcnRlcl9maWxlX2Rhc2hlclYw, mimeType: app

In [7]:
# '目录 id' in parents
file_list = drive.ListFile({'q': "'16zfnt8Cnuznzwj2RHGrxvyfyA6iQPRXx' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s, mimeType: %s' % (file1['title'], file1['id'], file1["mimeType"]))

title: atec_nlp_sim_train.csv, id: 1py3JAFOLOH2lM-lOqYyP_LfqaImlOqL_, mimeType: text/csv


In [0]:
file = drive.CreateFile({'id': "1py3JAFOLOH2lM-lOqYyP_LfqaImlOqL_"}) 
#这里的下载操作只是缓存，不会在你的Google Drive 目录下多下载一个文件
file.GetContentFile('data.csv', "text/csv") 

In [0]:
df = pd.read_csv('data.csv', sep='\t', names=['number', "sen1", "sen2", "label"], skipinitialspace=True)

In [0]:
def normalize_string(s):
    s = re.sub(r"\s+", r" ", s).strip()
    s = re.sub(r"\d+", r"N", s).strip()
    return s
  
  
flatten = lambda l: [item for sublist in l for item in sublist]

In [11]:
%%time

X1_r = list(map(normalize_string, df.sen1.tolist()))
X2_r = list(map(normalize_string, df.sen2.tolist()))
y_r = df.label.tolist()

for sen1, sen2, y in zip(X1_r, X2_r, y_r):
  if len(sen1) > MAX_LENGTH or len(sen2) > MAX_LENGTH:
    X1_r.remove(sen1)
    X2_r.remove(sen2)
    y_r.remove(y)
    
    X1_r.append(sen1[:MAX_LENGTH])
    X2_r.append(sen2[:MAX_LENGTH])
    y_r.append(y)
    
print(len(X1_r), len(X2_r), len(y_r))
print(X1_r[0], "@@@@", X2_r[0], "@@@@",y_r[0])

39346 39346 39346
﻿怎么更改花呗手机号码 @@@@ 我的花呗是以前的手机号码，怎么更改成现在的支付宝的号码手机号 @@@@ 1
CPU times: user 484 ms, sys: 0 ns, total: 484 ms
Wall time: 492 ms


In [12]:
vocab = list(set(flatten(X1_r + X2_r)))
len(vocab)

1701

In [13]:
source2index = {'<PAD>':0,'<UNK>':1,'<s>':2,'</s>':3}
for vo in vocab:
    if vo not in source2index.keys():
        source2index[vo]=len(source2index)
index2source = {v:k for k,v in source2index.items()}

vocab = source2index.keys()
len(vocab)

1705

In [0]:
def getBatch(batch_size,train_data):
    random.shuffle(train_data)
    sindex=0
    eindex=batch_size
    while eindex < len(train_data):
        batch = train_data[sindex:eindex]
        temp = eindex
        eindex = eindex+batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [0]:
def prepare_sequence(seq, to_index):
    idxs = list(map(lambda w: to_index[w] if w in to_index.keys() else to_index["<unk>"], seq))
    return LongTensor(idxs)

In [16]:
%%time
X1_p, X2_p = [],[]
ta_p = []

for s1, s2, ta in zip(X1_r, X2_r, y_r):
    X1_p.append(prepare_sequence(s1,source2index).view(1,-1))
    X2_p.append(prepare_sequence(s2,source2index).view(1,-1))
    ta_p.append(ta)
    
data = list(zip(X1_p, X2_p, ta_p))
np.random.shuffle(data)
train_data = data[:30000]
validate_data = data[30000:35000]
test_data = data[35000:]

CPU times: user 720 ms, sys: 13 ms, total: 733 ms
Wall time: 734 ms


In [17]:
np.sum(np.array([i[2] for i in validate_data]) == 1)

1087

In [18]:
np.sum(np.array([i[2] for i in validate_data]) == 0)

3913

In [0]:
def pad_to_batch(batch,x_to_ix):
    
    sorted_batch =  sorted(batch, key=lambda b:b[0].size(1),reverse=True) # sort by len
    
    x1, x2, y = list(zip(*sorted_batch))
    max_x1 = max([s.size(1) for s in x1])
    max_x2 = max([s.size(1) for s in x2])
    
    x1_p, x2_p, y_p = [],[],[]
    for i in range(len(batch)):
      
        if x1[i].size(1)<max_x1:
            x1_p.append(torch.cat( [ Variable(x1[i]),Variable(LongTensor([x_to_ix['<PAD>']]*(max_x1-x1[i].size(1)))).view(1,-1) ],1))
        else:
            x1_p.append(Variable(x1[i]))
        
        if x2[i].size(1)<max_x2:
            x2_p.append(torch.cat( [ Variable(x2[i]),Variable(LongTensor([x_to_ix['<PAD>']]*(max_x2-x2[i].size(1)))).view(1,-1) ],1))
        else:
            x2_p.append(Variable(x2[i]))
        
    x1_var = torch.cat(x1_p, 0)
    x2_var = torch.cat(x2_p, 0)
    target_var = Variable(LongTensor(y))
    x1_len = [list(map(lambda s: s ==0, t.data)).count(False) for t in x1_var]
    x2_len = [list(map(lambda s: s ==0, t.data)).count(False) for t in x2_var]
    
    return x1_var, x2_var, target_var, x1_len, x2_len

In [0]:
EPOCH=150
BATCH_SIZE = 128
EMBEDDING_SIZE = 128
LR = 0.01
DECODER_LEARNING_RATIO=5.0
RESCHEDULED=False

In [21]:
batch = next(getBatch(BATCH_SIZE,train_data))
pbatch = pad_to_batch(batch,source2index)
pbatch[0]

Variable containing:
  830  1447  1212  ...   1212  1152   611
 1497   803   405  ...    398     0     0
  217   836  1683  ...      0     0     0
       ...          ⋱          ...       
  405   442   825  ...      0     0     0
  405   442  1034  ...      0     0     0
  405   442  1346  ...      0     0     0
[torch.LongTensor of size 128x40]

In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score

def train_perf():
  
  print("========== performance on randomly selected 5000 samples from training data")
  samples = random.sample(train_data, 5000)
  sen1, sen2, targets,sen1_lengths, sen2_lengths = pad_to_batch(samples,source2index)
  outputs = model(sen1, sen2, sen1_lengths, sen2_lengths)
  t_predict = np.array(np.argmax(outputs.data, 1))
  t_targets = np.array(targets.data)
  print("Correct: ", np.sum(t_targets == t_predict))
  print("tn, fp, fn, tp \n", confusion_matrix(t_targets, t_predict).ravel())
  print("precsion: ", precision_score(t_predict, t_targets))
  print("recall: ", recall_score(t_predict, t_targets))
  print("f1 score: ", f1_score(t_predict, t_targets))
  
def validate_perf():
  
  print("========== performance on validation data")
  sen1, sen2, targets,sen1_lengths, sen2_lengths = pad_to_batch(validate_data,source2index)
  outputs = model(sen1, sen2, sen1_lengths, sen2_lengths)
  predict = np.array(np.argmax(outputs.data, 1))
  targets = np.array(targets.data)
  print("Correct: ", np.sum(predict == targets))
  print("tn, fp, fn, tp \n", confusion_matrix(targets, predict).ravel())
  print("precsion: ", precision_score(predict, targets))
  print("recall: ", recall_score(predict, targets))
  print("f1 score: ", f1_score(predict, targets))  

In [0]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [0]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [0]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [0]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [0]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        #return x + self.dropout(sublayer(self.norm(x)))  # this is not LayerNorm(x+Sublayer(x))
        return self.norm(x + self.dropout(sublayer(x)))

In [0]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [0]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [0]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, hidden_size, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert hidden_size % h == 0
        # We assume d_v always equals d_k
        self.d_k = hidden_size // h         # we employ h=4 parallel attention layers, 8//4=2
        self.h = h
        self.linears = clones(nn.Linear(hidden_size, hidden_size), 4) #2nd d_model should be dk or dv, WQ, WK, WV, WO
        #self.output = nn.Linear(hidden_size * h, hidden_size)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        #print("nbatches", nbatches)
        
        #print("query, key, value - 1", query, key, value)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k = d_model(hidden_size), hidden 被拆分为3部分
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        #print("query, key, value - 2", query, key, value)
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [0]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))  ## k, q, v = x, x, x
        return self.sublayer[1](x, self.feed_forward)

In [0]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N, vocab_size, hidden_size, position):
        super(Encoder, self).__init__()
        self.embedding = nn.Sequential(Embeddings(hidden_size, vocab_size), position)
        
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        x = self.embedding(x)
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [0]:
class Model(nn.Module):
    
    def __init__(self, encoder, kernel_num=100, kernel_sizes=[3,4,5], dropout=0.1):
        
        super(Model, self).__init__()
        self.encoder = encoder
        
        self.convs = nn.ModuleList([nn.Conv2d(1, kernel_num, (K, EMBEDDING_SIZE)) for K in kernel_sizes])

        # kernal_size = (K,D) 
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(len(kernel_sizes)*kernel_num, 2)
        
    def init_weight(self):
        pass
        
    def forward(self, sen1, sen2, sen1_lengths, sen2_lengths, is_training=False):
             
        outputs_1 = self.encoder(sen1,None)
        outputs_2 = self.encoder(sen2,None)
        
        #print("outputs_1: ", outputs_1.shape)
        #print("outputs_2: ", outputs_2.shape)
        inputs = torch.cat((outputs_1, outputs_2), 1)
        #print("inputs: ", inputs.shape)
        
        inputs = torch.unsqueeze(inputs, 1)
        #print("inputs: ", inputs.shape)
        
        inputs = [F.relu(conv(inputs)).squeeze(3) for conv in self.convs] #[(N,Co,W), ...]*len(Ks)
        inputs = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in inputs] #[(N,Co), ...]*len(Ks)

        concated = torch.cat(inputs, 1)

        if is_training:
            concated = self.dropout(concated) # (N,len(Ks)*Co)
        out = self.fc(concated) 
        
        return out

In [0]:
def make_model(vocab_size, N=2, 
               hidden_size=8, d_ff=4, h=2, dropout=0.1):
  
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, hidden_size)
    ff = PositionwiseFeedForward(hidden_size, d_ff, dropout)
    position = PositionalEncoding(hidden_size, dropout)
    
    el = EncoderLayer(hidden_size, c(attn), c(ff), dropout)
    encoder = Encoder(el, N, vocab_size, hidden_size, c(position))
    model = Model(encoder)
    model.init_weight()

    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [0]:
alpha = Variable(FloatTensor([0.75, 0.25]))  # increase loss weights for positive samples
gamma = 2

def one_hot(index, classes):
    size = index.size() + (classes,)
    view = index.size() + (1,)

    mask = FloatTensor(*size).fill_(0)
    index = index.view(*view)
    ones = 1.

    if isinstance(index, Variable):
        ones = Variable(FloatTensor(index.size()).fill_(1))
        mask = Variable(mask, volatile=index.volatile)

    return mask.scatter_(1, index, ones)


class FocalLoss(nn.Module):

    def __init__(self, gamma=0, eps=1e-7):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.eps = eps

    def forward(self, input, target):
        y = one_hot(target, input.size(-1))
        logit = F.softmax(input, dim=-1)
        logit = logit.clamp(self.eps, 1. - self.eps)   # pt

        loss = -1 * y * torch.log(logit) * alpha # cross entropy
        loss = loss * (1 - logit) ** self.gamma # focal loss

        return loss.sum()

In [0]:
# Small example model.
EMBEDDING_SIZE = 64
LR = 0.01

model = make_model(len(vocab), 2, EMBEDDING_SIZE)


In [36]:
model.parameters

<bound method Module.parameters of Model(
  (encoder): Encoder(
    (embedding): Sequential(
      (0): Embeddings(
        (lut): Embedding(1705, 64)
      )
      (1): PositionalEncoding(
        (dropout): Dropout(p=0.1)
      )
    )
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0): Linear(in_features=64, out_features=64)
            (1): Linear(in_features=64, out_features=64)
            (2): Linear(in_features=64, out_features=64)
            (3): Linear(in_features=64, out_features=64)
          )
          (dropout): Dropout(p=0.1)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=64, out_features=4)
          (w_2): Linear(in_features=4, out_features=64)
          (dropout): Dropout(p=0.1)
        )
        (sublayer): ModuleList(
          (0): SublayerConnection(
            (norm): LayerNorm(
            )
            (dropout): Dropout

In [0]:
#criterion = nn.CrossEntropyLoss()
criterion = FocalLoss(2)
optimizer = optim.Adam(model.parameters(),lr=LR)

In [41]:
# batch * length * embedding_size

for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
    
    # sen1 = batch * l
    sen1, sen2, targets,sen1_lengths, sen2_lengths = pad_to_batch(batch,source2index)
    print("sen1 size: ", sen1.shape)
    
    # Forward + Backward + Optimize
    optimizer.zero_grad()
    outputs = model(sen1, sen2, sen1_lengths, sen2_lengths, True)
    print('outputs', outputs.shape)
    print('targets', targets.shape)
    loss = criterion(outputs, targets)
    print('loss', loss)
    loss.backward()
    optimizer.step()
    
    break
    
train_perf()
validate_perf()

sen1 size:  torch.Size([128, 33])
outputs torch.Size([128, 2])
targets torch.Size([128])
loss Variable containing:
 100.7381
[torch.FloatTensor of size 1]

========== performance on randomly selected 5000 samples from training data
Correct:  3892
tn, fp, fn, tp 
 [3892    0 1108    0]
precsion:  0.0
recall:  0.0
f1 score:  0.0
========== performance on validation data


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


Correct:  3913
tn, fp, fn, tp 
 [3913    0 1087    0]
precsion:  0.0
recall:  0.0
f1 score:  0.0


In [0]:
total_step = round(len(train_data) / BATCH_SIZE)

for epoch in range(EPOCH):
    
    total_loss = []
    
    for i,batch in enumerate(getBatch(BATCH_SIZE,train_data)):
    
      # sen1 = batch * l
      sen1, sen2, targets,sen1_lengths, sen2_lengths = pad_to_batch(batch,source2index)

      # Forward + Backward + Optimize
      optimizer.zero_grad()
      outputs = model(sen1, sen2, sen1_lengths, sen2_lengths, True)
      loss = criterion(outputs, targets)
      total_loss.append(loss.data[0])
      loss.backward()
      optimizer.step()

      if (i+1) % 20 == 0:
          print ('Epoch [{}/{}], Step [{}/{}], Mean Loss: {:.4f}' 
                     .format(epoch+1, EPOCH, i+1, total_step, np.mean(total_loss)))
          total_loss = []
    
    if (epoch + 1) %10 == 0:
        train_perf()
        validate_perf()
    
    #if RESCHEDULED == False and epoch  == EPOCH//4:
    if (epoch + 1) %25 == 0:
        LR *= 0.1
        print("LR now is: ", LR)
        optimizer = optim.Adam(model.parameters(), lr=LR)

Epoch [1/150], Step [20/234], Mean Loss: 115.6473
Epoch [1/150], Step [40/234], Mean Loss: 116.2517
Epoch [1/150], Step [60/234], Mean Loss: 115.4458
Epoch [1/150], Step [80/234], Mean Loss: 113.0281
Epoch [1/150], Step [100/234], Mean Loss: 110.6104
Epoch [1/150], Step [120/234], Mean Loss: 111.8193
Epoch [1/150], Step [140/234], Mean Loss: 114.6399
Epoch [1/150], Step [160/234], Mean Loss: 107.9912
Epoch [1/150], Step [180/234], Mean Loss: 111.0134
Epoch [1/150], Step [200/234], Mean Loss: 110.4089
Epoch [1/150], Step [220/234], Mean Loss: 106.5809
Epoch [2/150], Step [20/234], Mean Loss: 113.6325
Epoch [2/150], Step [40/234], Mean Loss: 114.6399
Epoch [2/150], Step [60/234], Mean Loss: 112.2222
Epoch [2/150], Step [80/234], Mean Loss: 113.2296
Epoch [2/150], Step [100/234], Mean Loss: 111.0134
Epoch [2/150], Step [120/234], Mean Loss: 111.4163
Epoch [2/150], Step [140/234], Mean Loss: 111.0134
Epoch [2/150], Step [160/234], Mean Loss: 111.8193
Epoch [2/150], Step [180/234], Mean Los